In [1]:
print("NER Trainer Runner Start!")
import pymongo
import datetime
import os
import subprocess
import signal
import time
from core.config import MONGODB_URL, DATABASE_NAME, CONFIG_COLLECTION, LABEL_RETRAIN_QUEUE_COLLECTION, API_PORT, API_HOST, API_WORKER, SLEEP_INTERVAL_SECOND, ANACONDA_ENV_NAME, PATH, NER_ADAPTERS_TRAINER_NAME

client = pymongo.MongoClient(MONGODB_URL)

from utils.logs import config_log, change_service_status

config_col = client[DATABASE_NAME][CONFIG_COLLECTION]

trainer = config_col.find_one({"name": NER_ADAPTERS_TRAINER_NAME})

if not trainer:
    trainer_obj = {
        "name": NER_ADAPTERS_TRAINER_NAME,
        "status": "down",
        "restart_required": False,
        "last_data_cache_timestamp": datetime.datetime.strptime("1999", "%Y"),
        "logs": [],
    }
    config_col.insert_one(trainer_obj)

def run_trainer():
    return_status = "Finish."
    command = f"""eval "$(conda shell.bash hook)";
        conda activate {ANACONDA_ENV_NAME};
        python {PATH}/NER_trainer.py;"""
    config_log(NER_ADAPTERS_TRAINER_NAME, "Start {NER_ADAPTERS_TRAINER_NAME} from runner")
    change_service_status(NER_ADAPTERS_TRAINER_NAME, "up")
    process = subprocess.Popen(command, shell=True)
    while True:
        try:
            trainer = config_col.find_one({
                "name": NER_ADAPTERS_TRAINER_NAME
            }, {"restart_required": True})
            print(trainer["restart_required"])
            if trainer["restart_required"]:
                config_col.update_one({
                    "name": NER_ADAPTERS_TRAINER_NAME
                },{
                    "$set": {"restart_required": False}
                })
                raise Exception("Restart Required") 
            try:
                poll = process.poll()
                int(poll)
                if poll == 0:
                    return_status = f"Finish with code {poll}"
                if poll == 4:
                    return_status = f"Error with code {poll}, CUDA out of memory!"
                    time.sleep(600) #if out of memory, don't keep try it...
                else:
                    return_status = f"Error with code {poll}"
                #print(f"Process Stop Running, return code {poll}")
                break
            except TypeError:
                #print("Process Still Running")
                #time.sleep(SLEEP_INTERVAL_SECOND)
                time.sleep(3)
                continue
            
        except KeyboardInterrupt:
            print("KeyboardInterrupt, stop program")
            return_status = "KeyboardInterrupt"
            break
        except Exception as e:
            return_status = "Trainer Restart Required."
            print(return_status)
            break

    config_log("trainer", f"Stop {NER_ADAPTERS_TRAINER_NAME} because {return_status}")
    change_service_status(NER_ADAPTERS_TRAINER_NAME, "down")
    #process.send_signal(signal.SIGINT)
    print("process.pid")
    os.kill(process.pid, signal.SIGINT)
    os.kill(process.pid, signal.SIGINT)
    os.kill(process.pid, signal.SIGINT)
    os.kill(process.pid, signal.SIGINT)
    os.kill(process.pid, signal.SIGINT)
    process.wait()
    print(f"Stop {NER_ADAPTERS_TRAINER_NAME} because {return_status}")
    return return_status

training_queue_col = client[DATABASE_NAME][LABEL_RETRAIN_QUEUE_COLLECTION]
while True:
    print("QQQ")
    import re
    result = training_queue_col.find({
        "status":  re.compile("(training|waiting)")
    })

    result = list(result)

    if result:
        return_status = run_trainer()
        if return_status == "KeyboardInterrupt":
            break
        else:
            continue
    time.sleep(SLEEP_INTERVAL_SECOND)

NER Trainer Runner Start!
QQQ
False
KeyboardInterrupt, stop program
process.pid
Stop NER_adapter_trainer because KeyboardInterrupt


Traceback (most recent call last):
  File "/home/eason/Python/GSoC_Accord/API/NER_trainer.py", line 8, in <module>
    import torch
  File "/home/eason/anaconda3/envs/adapter/lib/python3.7/site-packages/torch/__init__.py", line 196, in <module>
    from torch._C import *
RuntimeError: KeyboardInterrupt: 


In [36]:
import os
cwd = os.getcwd()

 33%|███▎      | 15808/47960 [02:48<10:46, 49.76it/s]

In [37]:
cwd

 33%|███▎      | 15814/47960 [02:48<10:42, 50.00it/s]

'/home/eason/Python/GSoC_Accord/API'

 33%|███▎      | 15964/47960 [02:51<10:42, 49.82it/s]

In [39]:
os.getpid()

 34%|███▍      | 16327/47960 [02:59<10:51, 48.54it/s]

22623

 35%|███▍      | 16577/47960 [03:04<10:52, 48.12it/s]

In [24]:
command = f"""eval "$(conda shell.bash hook)";
    conda activate {ANACONDA_ENV_NAME};
    python {PATH}/NER_trainer.py;"""
config_log(NER_ADAPTERS_TRAINER_NAME, "Start {NER_ADAPTERS_TRAINER_NAME} from runner")
change_service_status(NER_ADAPTERS_TRAINER_NAME, "up")
process = subprocess.Popen(command, shell=True)

Reading Data from MongoDB...


  2%|▏         | 1146/47960 [00:02<01:22, 570.52it/s]

In [28]:
process.pid

 21%|██        | 10163/47960 [01:13<08:18, 75.86it/s]

25788

 22%|██▏       | 10379/47960 [01:16<08:28, 73.91it/s]

In [35]:
process.stdout

 33%|███▎      | 15791/47960 [02:48<10:42, 50.06it/s]

In [44]:
client = pymongo.MongoClient(MONGODB_URL)
training_queue_col = client[DATABASE_NAME][LABEL_RETRAIN_QUEUE_COLLECTION]
training_queue = training_queue_col.find({
    "status":  re.compile("(training|waiting)")
})
training_queue = list(training_queue)

if len(training_queue) == 0:
    sys.exit(0)

# Old Train First
training_queue.sort(key = lambda x: x["add_time"], reverse= False)

now_is_training = training_queue[0]

In [19]:
from subprocess import signal
os.kill(process.pid, signal.SIGINT)

 22%|██▏       | 10767/47960 [01:21<08:40, 71.40it/s]

In [ ]:
import pymongo
client = pymongo.MongoClient(MONGODB_URL)
training_queue_col = client[DATABASE_NAME][LABEL_RETRAIN_QUEUE_COLLECTION]

training_queue = training_queue_col.find({
    "status":  re.compile("(training|waiting)")
})
training_queue = list(training_queue)

while True:
    if True or len(training_queue) == 0:
        break

# Old Train First
training_queue.sort(key = lambda x: x["add_time"], reverse= False)

now_is_training = training_queue[0]

# Update parameter On Each Iter
train_data_search_filter = now_is_training["train_data_filter"]
label_name = now_is_training["label_name"]
Epoch_Times = now_is_training["epochs"]

In [ ]:
from utils.logs import trainer_log, queue_task_log

In [ ]:
training_queue_col = client[DATABASE_NAME][LABEL_RETRAIN_QUEUE_COLLECTION]
import re
result = training_queue_col.find({
    "status": {
        "$not": re.compile("done")}
})
result = list(result)

In [ ]:
type(result[0]["_id"])

In [ ]:
import os
import subprocess
import signal
import time

In [ ]:
command = "python trainer.py"
process = subprocess.Popen(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

In [ ]:
process.poll()

In [ ]:
# stop process
process.send_signal(signal.SIGINT)

In [ ]:
process.kill()

In [ ]:
type(process)

In [ ]:
process.pid

In [ ]:
while True:
    time.sleep(1)

In [ ]:
process